# Introduction to funcX

funcX is a Function-as-a-Service (FaaS) platform for science that enables you to convert almost any resource into a high-performance function serving device. Deploying a funcX endpoint will integrate your resource into the function serving fabric, allowing you to dynamically send, monitor, and receive results from function invocations. funcX is underpinned by Parsl, allowing you to connect your endpoint to large compute resources via traditional batch queues, where funcX will dynamically provision, use, and release resources on-demand to fulfill function requests.

Here we provide an example of using funcX to register a function and run it on a publicly available tutorial endpoint.

We start by creating a funcX client to interact with the service.

In [1]:
import json
import sys

from funcx.sdk.client import FuncXClient

In [2]:
fxc = FuncXClient()

Here we define the tutorial endpoint to be used in this demonstration. Because the tutorial endpoint is Kubernetes-based, we select a simple python3.6 container that will be used during execution.

In [3]:
endpoint_uuid = 'a92945a1-2778-4417-8cd1-4957bc35ce66' 
container_uuid = '3861862b-152e-49a4-b15e-9a5da4205cad'

Now we define a little hello-world function and register it with the service.

In [4]:
func = """
def hello_world(event):
    data = event['data']
    if len(data) > 0:
        return f"Hello, {data}!"
    return "Hello, world!"
"""

# Register the function
func_uuid = fxc.register_function("hello_world", func, "hello_world", 
                                  description="A test function.",
                                  container=container_uuid)
func_uuid

'6084e6ba-9309-4914-b1d1-d29b455d388a'

Now we can define an input to the function and run it.

Setting asynchronous to True means the call will be non-blocking and will return a future that can be used to check the status of the execution and retrieve the result.

In [5]:
payload = ""
res = fxc.run(payload, endpoint_uuid, func_uuid, asynchronous=True, async_poll=5)
res

<FuncXFuture at 0x7f6218ec99e8 state=running>

Retrieve the result. Note, it may take a minute for the tutorial endpoint to spin up a resource to run your function.

In [6]:
res.result()

'Hello, world!'

Try it with some other input!

In [7]:
payload = "Ryan"
res = fxc.run(payload, endpoint_uuid, func_uuid, asynchronous=True, async_poll=5)
res.result()

'Hello, Ryan!'